# Reverse numerical methods applied to channel flow

Our goals here, in general, are to find new valid equations that describe fluid flow. 
We will try random PDEs and use the RANS equations to attempt to solve for the Reynolds
stresses in terms of the other quantities we know. 

Navier--Stokes:

$$
\frac{\partial u}{\partial t} + u \frac{\partial u}{\partial x} 
+ v \frac{\partial u}{\partial y} + w \frac{\partial u}{\partial z}
= - \frac{1}{\rho}\frac{\partial p}{\partial x} + \nu \left( \frac{\partial^2 u}{\partial x^2}
+ \frac{\partial^2 u}{\partial y^2} + \frac{\partial^2 u}{\partial z^2} \right)
$$

RANS:

$$
\frac{\partial U}{\partial t} + U \frac{\partial U}{\partial x} 
+ V \frac{\partial U}{\partial y} + W \frac{\partial U}{\partial z}
= - \frac{1}{\rho}\frac{\partial P}{\partial x} + \nu \left( \frac{\partial^2 U}{\partial x^2}
+ \frac{\partial^2 U}{\partial y^2} + \frac{\partial^2 U}{\partial z^2} \right)
- \left( \frac{\partial}{\partial x} \overline{u'u'}
+ \frac{\partial}{\partial y} \overline{u'v'}
+ \frac{\partial}{\partial z} \overline{u'w'} \right)
$$


$$
\frac{\partial \vec{U}}{\partial t} + (\vec{U} \cdot \nabla) \vec{U}
= - \frac{1}{\rho} \nabla P + \nu \nabla^2 \vec{U} - \nabla \cdot \vec{\vec{R}}
$$

## Steady RANS equations with generic Reynolds stress effects

$$
(\vec{U} \cdot \nabla) \vec{U}
+ \frac{1}{\rho} \nabla P 
- \nu \nabla^2 \vec{U}
= \mathbf{R},
$$

where in this case $\mathbf{R}$ is simply the effects of the Reynolds stresses (i.e., the opposite of the gradient), not the Reynolds stresses themselves.

Now, let's add some terms and assume that their coefficients are isotropic:

$$
(\vec{U} \cdot \nabla) \vec{U}
+ \frac{1}{\rho} \nabla P 
- \nu \nabla^2 \vec{U}
= 
  A (\nabla P)^2
+ B \nabla P^2
+ C (\vec{U} \cdot \nabla) \vec{U}^2
+ D \nabla^2 \vec{U}^2
+ E \nabla K
$$

$$
U \frac{\partial U}{\partial x} 
+ V \frac{\partial U}{\partial y} + W \frac{\partial U}{\partial z}
= - \frac{1}{\rho}\frac{\partial P}{\partial x} + \nu \left( \frac{\partial^2 U}{\partial x^2}
+ \frac{\partial^2 U}{\partial y^2} + \frac{\partial^2 U}{\partial z^2} \right)
+ R_x
$$

$$
U \frac{\partial V}{\partial x} 
+ V \frac{\partial V}{\partial y} + W \frac{\partial V}{\partial z}
= - \frac{1}{\rho}\frac{\partial P}{\partial y} + \nu \left( \frac{\partial^2 V}{\partial x^2}
+ \frac{\partial^2 V}{\partial y^2} + \frac{\partial^2 V}{\partial z^2} \right)
+ R_y
$$

$$
U \frac{\partial W}{\partial x} 
+ V \frac{\partial W}{\partial y} + W \frac{\partial W}{\partial z}
= - \frac{1}{\rho}\frac{\partial P}{\partial z} + \nu \left( \frac{\partial^2 W}{\partial x^2}
+ \frac{\partial^2 W}{\partial y^2} + \frac{\partial^2 W}{\partial z^2} \right)
+ R_z
$$

## OpenFOAM's momentum equation

$$
\frac{\partial \vec{U}}{\partial t}
+ \nabla \cdot \phi \vec{U}
- \nu_{\mathrm{eff}} \nabla^2 \vec{U}
- \nabla \cdot \nu_{\mathrm{eff}} \mathrm{dev}(\nabla \vec{U}^T ) 
= - \frac{1}{\rho} \nabla P
$$


## `simpleFoam` momentum equation


```C++
tmp<fvVectorMatrix> UEqn
(
    fvm::div(phi, U)
  + turbulence->divDevReff(U)
  ==
    fvOptions(U)
);

UEqn().relax();

fvOptions.constrain(UEqn());

solve(UEqn() == -fvc::grad(p));

fvOptions.correct(U);
```


## Spalart--Allmaras turbulence model `divDevReff`


```C++
tmp<fvVectorMatrix> SpalartAllmaras::divDevReff(volVectorField& U) const
{
    const volScalarField nuEff_(nuEff());

    return
    (
      - fvm::laplacian(nuEff_, U)
      - fvc::div(nuEff_*dev(T(fvc::grad(U))))
    );
}
```

## Launder--Gibson Reynolds stress model `divDevReff`

```C++
tmp<fvVectorMatrix> LaunderGibsonRSTM::divDevReff(volVectorField& U) const
{
    if (couplingFactor_.value() > 0.0)
    {
        return
        (
            fvc::div(R_ + couplingFactor_*nut_*fvc::grad(U), "div(R)")
          + fvc::laplacian
            (
                (1.0 - couplingFactor_)*nut_,
                U,
                "laplacian(nuEff,U)"
            )
          - fvm::laplacian(nuEff(), U)
        );
    }
    else
    {
        return
        (
            fvc::div(R_)
          + fvc::laplacian(nut_, U, "laplacian(nuEff,U)")
          - fvm::laplacian(nuEff(), U)
        );
    }
}
```

In [ ]:
import numpy as np
import pyJHTDB
from pyJHTDB.dbinfo import channel as info

npoints = 128
nskip = 4

# x = np.random.random(size=(npoints, info['ny']/nskip-1, 3)).astype(np.float32)

# x = np.ones((1, 3))*0.5
x = [(0.5, 0.0, 0.5)]

# x[..., 0] *= info['lx']
# x[..., 1] = 0
# # x[..., 1] *= info['dy'][::nskip][None, :x.shape[1]]
# # x[..., 1] += info['ynodes'][::nskip][None, :x.shape[1]]
# x[..., 2] *= info['lz']

print(info["xnodes"].min(), info["xnodes"].max(),
      info["ynodes"].min(), info["ynodes"].max(),
      info["znodes"].min(), info["znodes"].max()
     )

x = np.asarray(x, dtype=np.float32)
time = np.linspace(0, 10.0, 30)

Now I'm actually calling the database, and requesting the velocity gradient.
Afterwards, I'm averaging over the first axis of x, since the y dependency is in the second axis.
For now, only M1Q4, M2Q8 and M2Q14 are implemented for the channel dataset, therefore they're the only ones I'm using

In [ ]:
from pyJHTDB import libJHTDB

# interp_info = [(44, 'FD4Lag4'),
#                (104, 'M1Q4'),
#                (208, 'M2Q8'),
#                (214, 'M2Q14')]
# divu = []

lJHTDB = libJHTDB()
lJHTDB.initialize()

"""
for ii in interp_info:
    gradu = lJHTDB.getData(
                0.0,
                x,
                sinterp = ii[0],
                tinterp = 0,
                data_set = info['name'],
                getFunction = 'getVelocityGradient')
    divu.append(np.average(np.abs(
                        gradu[..., 0] +
                        gradu[..., 4] +
                        gradu[..., 8]), axis = 0) / 
                np.average(np.sqrt(
                        gradu[..., 0]**2 +
                        gradu[..., 4]**2 +
                        gradu[..., 8]**2), axis = 0))
"""

u = []

for t in time:
    u.append(lJHTDB.getData(t, x, getFunction="getVelocity", tinterp="PCHIPInt",
                            sinterp=104, data_set=info["name"]))

lJHTDB.finalize()

In [ ]:
u = np.asarray(u)
u = u[:, 0]
print(u.shape)


In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

fig = plt.figure(figsize = (10, 6))
ax = fig.add_subplot(111)

# for i in range(len(divu)):
#     ax.plot(info['ynodes'][::nskip][:divu[i].shape[0]], divu[i], label=interp_info[i][1])

# ax.set_ylabel("Divergence")
# ax.legend(loc='best')
# ax.set_ylim(0, 0.255)

for n, label in enumerate(["$u$", "$v$", "$w$"]):
    plt.plot(time, u[:, n], label=label)
plt.xlabel("Time (s)")
plt.ylabel("Velocity")
plt.legend(loc="best")
plt.show()